# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import pandas as pd
from unidecode import unidecode

networks_url = "http://api.citybik.es/v2/networks"
response = requests.get(networks_url)
data = response.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [2]:
vitoria_network = None
for network in data["networks"]:
    city = unidecode(network["location"]["city"].lower())
    if "vitoria" in city:
        vitoria_network = network
        print("Found network:", network["id"], "-", network["location"]["city"])

Found network: bikevitoria - Vitória


Put your parsed results into a DataFrame.

In [3]:
if vitoria_network:
    network_id = vitoria_network["id"]
    stations_url = f"http://api.citybik.es/v2/networks/{network_id}"
    stations_response = requests.get(stations_url)

    if stations_response.status_code == 200:
        stations_data = stations_response.json()
        stations = stations_data['network']['stations']

        parsed = [
            {
                "name": station["name"],
                "latitude": station["latitude"],
                "longitude": station["longitude"],
                "free_bikes": station["free_bikes"]
            }
            for station in stations
        ]

        df = pd.DataFrame(parsed)
        pd.set_option('display.max_columns', None)
        print(df[["name", "free_bikes", "latitude", "longitude"]])
        print("Total stations:", len(df))
    else:
        print("Failed to retrieve station data.")
else:
    print("Vitoria network not found.")

                                          name  free_bikes   latitude  \
0              Mercado da Vila Rubim - Vitória           2 -20.321076   
1            Santo Antônio - Prainha - Vitória           1 -20.311602   
2                  Fábrica de Idéias - Vitória           6 -20.313714   
3                      Reta da Penha - Vitória           0 -20.291052   
4          SOE Jardim Camburi - Kids - Vitória           7 -20.266852   
5                  Praça dos Desejos - Vitória           2 -20.303819   
6                      SOE Beira Mar - Vitória           1 -20.318521   
7   Baia de Skate - Praia de Camburi - Vitória           1 -20.272822   
8    Prefeitura Municipal de Vitória - Vitória           0 -20.317976   
9                     SICOOB Central - Vitória           2 -20.303622   
10                        Goiabeiras - Vitória           3 -20.265986   
11                     Praça do Papa - Vitória           0 -20.316604   
12                           Itararé - Vitória     

In [ ]:
df_stations = df 
df_stations.to_csv("bike_stations_vitoria.csv", index=False)